In [53]:
import graphlab
# Set product key on this computer. After running this cell, you will not need to re-enter your product key. 
graphlab.product_key.set_product_key('6F36-4F68-0B23-4426-E305-5A92-90D6-0230')

# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

# Output active product key.
graphlab.product_key.get_product_key()

graphlab.canvas.set_target('ipynb')
sales = graphlab.SFrame('.\\Week 2\\home_data.gl/')

In [54]:
sales.column_names

<bound method SFrame.column_names of Columns:
	id	str
	date	datetime
	price	int
	bedrooms	str
	bathrooms	str
	sqft_living	int
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 21613

Data:
+------------+---------------------------+---------+----------+-----------+-------------+
|     id     |            date           |  price  | bedrooms | bathrooms | sqft_living |
+------------+---------------------------+---------+----------+-----------+-------------+
| 7129300520 | 2014-10-13 00:00:00+00:00 |  221900 |    3     |     1     |     1180    |
| 6414100192 | 2014-12-09 00:00:00+00:00 |  538000 |    3     |    2.25   |     2570    |
| 5631500400 | 2015-02-25 00:00:00+00:00 |  180000 |    2     |     1     |     770     |
| 2487200875 | 2014-12-09 00:00:00+00:00 |  604000 |    4     |     3     |     1960    |
| 19

In [55]:
#question1 find highest average house sale price

maxPrice = 0.0
maxArea = ''
zipcodeSet = set(sales['zipcode'])
zipcodeDict = {}
for zipcode in zipCodeSet:
    subData = sales[sales['zipcode'] == zipcode]
    avgPrice = subData['price'].mean()
    zipcodeDict[zipcode] = avgPrice
    if avgPrice > maxPrice:
        maxPrice = avgPrice
        maxArea = zipcode
    
print "zipCode:", maxArea, ",maxPrice:", maxPrice

zipCode: 98039 ,maxPrice: 2160606.6


In [56]:
#question2  fraction of the all houses have ‘sqft_living’ between (2000, 4000)
filterData = sales[(sales['sqft_living'] > 2000) & (sales['sqft_living'] < 4000)]
print 1.0 * filterData.num_rows() / sales.num_rows()

0.421551843798


In [57]:
trainData, testData = sales.random_split(0.8, seed = 0)


In [84]:
n1 = set(trainData['bedrooms'])
n2 = set(trainData['bathrooms'])
n3 = set(trainData['zipcode'])
n4 = set(trainData['floors'])
print len(n1)
print len(n2)
print len(n3)
print len(n4)
print "total coefficient numbers = (n1 + n2 + n3 + n4)(dummy variables) + intercept + sqft_living + sqft_lot - 4(independent required)"

my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']
regression1 = graphlab.linear_regression.create(trainData, target = 'price', features = my_features, validation_set=None)


Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.035451     | 3763208.270524     | 181908.848367 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

<bound method LinearRegression.summary of Class                          : LinearRegression

Schema
------
Number of coefficients         : 115
Number of examples             : 17384
Number of feature columns      : 6
Number of unpacked features    : 6

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0546

Settings
--------
Residual sum of squares        : 5.75250973324e+14
Training RMSE                  : 181908.8484

Highest Positive Coefficients
-----------------------------
bathrooms[8]                   : 1741315.8161
zipcode[98039]                 : 1162973.0381
bathrooms[5.75]                : 1145716.2212
bathrooms[6]                   : 1040376.2448
bathrooms[6.25]                : 919328.0218

Lowest Negative C

In [61]:
advanced_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode', 'condition', 'grade',
                     'waterfront', 'view', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long', 
                     'sqft_living15','sqft_lot15']
regression2 = graphlab.linear_regression.create(trainData, target = 'price', features = advanced_features, validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 18

Number of unpacked features : 18

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.083260     | 3469012.450663     | 154580.940735 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [63]:
print regression1.evaluate(testData)
print regression2.evaluate(testData)

{'max_error': 3486584.509381928, 'rmse': 179542.43331269105}
{'max_error': 3556849.413848093, 'rmse': 156831.11680191013}
